# Enmascaramiento de diferencia NDVI google Earth Engine

In [1]:
# Importar paquete GEE
import ee
ee.Initialize()

## Seleccionar imagenes de 20 años diferencia

In [2]:
# Seleccion dos imagenes diferencia 20 años
image1 = ee.Image("LANDSAT/LT05/C01/T1_TOA/LT05_044034_19900604")\
              .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7'],
                     ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2'])

In [3]:
image2 = ee.Image("LANDSAT/LT05/C01/T1_TOA/LT05_044034_20100611")\
              .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7'],
                     ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2'])

In [4]:
# Importar geemap
import geemap
Map = geemap.Map(basemap='ESRI')

In [23]:
geomatria = image1.geometry()
Map.centerObject(geomatria)

In [8]:
# Nombre de las bandas
print("Imagen1:",image1.bandNames().getInfo())
print("Imagen2:",image2.bandNames().getInfo())

Imagen1: ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']
Imagen2: ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']


In [9]:
# Simbologia escala 0 - 1 TOA
viz_es = {
    'bands': ['SWIR1','NIR','RED'],
    'min': 0.1,
    'max': 0.6,
    'gamma': 1.6
}

In [10]:
# Visualizar 
Map.addLayer(image1, viz_es, name = "Landsat8 IMG1")
Map.addLayer(image2, viz_es, name = "Landsat8 IMG2")
Map

Map(center=[37.482218868889554, -122.15074683989116], controls=(WidgetControl(options=['position', 'transparen…

## Calcular NDVI

In [11]:
# Determinar NDVI forma 3 Funcion GEE
def Calc_NDVI(image):
    ndvi = image.expression('float(nir - red)/(nir + red)', {
        'nir': image.select('NIR'),
        'red': image.select('RED')
    })

    return ndvi.rename("NDVI")

In [12]:
# Calcular indices con funcion
ndvi_1 = Calc_NDVI(image1)
ndvi_2 = Calc_NDVI(image2)

In [13]:
# Simbologia de NDVI
viz_ndvi = {
    'palette': ['FFFFFF', 'CE7E45','DF923D', 'F1B555',
                'FCD163', '99B718','74A901', '66A000',
                '529400', '3E8601','207401', '056201',
                '004C00', '023B01','012E01', '011D01',
                '011301'],
    'min': -0.4,
    'max': 1,
    'bands': "NDVI"
}

In [14]:
# Visualizar 
Map.addLayer(ndvi_1, viz_ndvi, name = "ndvi 1990")
Map.addLayer(ndvi_2, viz_ndvi, name = "ndvi 2010")
Map

Map(bottom=405983.0, center=[37.66996211213936, -121.43823623657228], controls=(WidgetControl(options=['positi…

## Diferencia de NDVI

In [15]:
# Diferencia de NDVI.
Diferencia_ndvi = ndvi_2.subtract(ndvi_1)

## Agregar MDE

In [16]:
# Agregar MDE SRTM Mascara
MDE = ee.Image("CGIAR/SRTM90_V4")
MDE_Mask = MDE.mask()

In [19]:
# Visualizar 
Map.addLayer(MDE_Mask,{},"MDE Mascara")
Map

Map(bottom=2381.0, center=[-2.8991526985043006, -62.35839843750001], controls=(WidgetControl(options=['positio…

## Actualizar enmascaramiento

In [20]:
# Actualizar mascara de NDVI mediante MDE
mask_ndvi = Diferencia_ndvi.updateMask(MDE_Mask)

In [21]:
# Simbologia de NDVI diferenciada
vizParams = {
    'palette': ["FF0000", "FFFFFF", "0000FF"],
    'min': -0.5,
    'max': 0.5
}

In [22]:
# Visualizar 
Map.addLayer(mask_ndvi, vizParams, name = "NDVI difference")
Map

Map(bottom=2381.0, center=[-2.8991526985043006, -62.35839843750001], controls=(WidgetControl(options=['positio…